# Это шаблон ноута для теста методов кластеризации

## imports

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score

## reading data files

In [2]:
# YOUR DATA DIRECTORY HERE
data_dir = '/media/arrteom/3923b309-fef1-47f0-a74c-cd259de5b45b/PAMAP2_Dataset/PAMAP2_Dataset/' 
data_path = os.path.join(data_dir, 'pamap_small.csv')

In [3]:
df_init = pd.read_csv(data_path)
df_init.shape

(248827, 41)

In [4]:
df_init = pd.read_csv(data_path)
print(f'full df shape {df_init.shape}')
df_train, df_test = train_test_split(df_init, test_size=0.3, random_state=42) # do not change
del df_init
print(f'{df_train.shape = }, {df_test.shape = }')

full df shape (248827, 41)
df_train.shape = (174178, 41), df_test.shape = (74649, 41)


## preprocessing
тут стоит в общих чертах написать про то почему такой препроцессинг

In [5]:
# add some real preprocessing logic instead of that
class PreprocessingPipeline:
    def __init__(self, prep_param=None):
        self.prep_param = prep_param
    
    def fit(self, X):
        pass
    
    def transform(self, X):
        return X.fillna(0)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    

In [6]:
prep_pipeline = PreprocessingPipeline()
X_train = prep_pipeline.fit_transform(df_train.drop(columns='activityID'))
y_train = df_train['activityID']

## cross validation param selection
тут немного про то в каких границах подбирали какие параметры и почему

In [45]:
from sklearn.cluster import KMeans


param_grid = {
    'n_clusters': [2, 3],  # тут нужно осмысленно выборать параметры
    'n_init': [5, 10]
}
grid_search = GridSearchCV(KMeans(), param_grid, cv=5, scoring='adjusted_mutual_info_score')

In [46]:
grid_search.fit(X_train, y_train)
print(grid_search.best_score_)

0.22858227084479643


## model train

In [47]:
full_model = KMeans(**grid_search.best_params_).fit(X_train, y_train)

## metrics

In [48]:
X_test = prep_pipeline.transform(df_test.drop(columns='activityID'))
y_test = df_test['activityID']
preds = full_model.predict(X_test)

In [49]:
train_armi = adjusted_mutual_info_score(y_train, full_model.predict(X_train))
test_ari = adjusted_rand_score(y_test, preds)
test_armi = adjusted_mutual_info_score(y_test, preds)

print(f'{test_ari = :.4f}, {test_armi = :.4f}, {train_armi = :.4f}')

test_ari = 0.0939, test_armi = 0.2289, train_armi = 0.2285


## clusters visualized

In [40]:
# TODO